In [4]:
%load_ext autoreload
%autoreload 2
import os
import sys
import yaml
import datetime
import numpy as np
import pandas as pd
from func.utils import get_categorical_features, to_pkl_gzip, read_pkl_gzip
from func.ml_utils import Classifier

HOME = os.path.expanduser('~')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
COLUMN_ID = 'TransactionID'
COLUMN_DT = 'TransactionDT'
COLUMN_TARGET = 'isFraud'
COLUMNS_IGNORE = [COLUMN_ID, COLUMN_DT, COLUMN_TARGET]

train_df = pd.read_csv('../input/train_transaction.csv')
test_df  = pd.read_csv('../input/test_transaction.csv')

COLUMNS_CATEGORY = get_categorical_features(train_df, COLUMNS_IGNORE)
use_cols = [col for col in train_df.columns if col not in COLUMNS_IGNORE+COLUMNS_CATEGORY]

In [8]:
# Save Base
test_df[COLUMN_TARGET] = np.nan
base_train = train_df[[COLUMN_ID, COLUMN_TARGET]]
base_test = test_df[[COLUMN_ID, COLUMN_TARGET]]

to_pkl_gzip(obj=base_train, path='../input/base_train')
to_pkl_gzip(obj=base_test, path='../input/base_test')

In [6]:
# Save Train Test Feature
for col in train_df.columns:
    feature = train_df[col].values
    to_pkl_gzip(obj=feature, path=f'../feature/raw_main/{col}_train')

for col in test_df.columns:
    feature = test_df[col].values
    to_pkl_gzip(obj=feature, path=f'../feature/raw_main/{col}_test')

In [37]:
COLUMN_DATATYPE = 'data_type'
train_df[COLUMN_DATATYPE] = 'train'
test_df[COLUMN_DATATYPE] = 'test'
rawdata = pd.concat([train_df, test_df], axis=0)
rawdata.to_csv(file_path, index=False)

In [15]:
from func.BigQuery import BigQuery

# BigQuery
dataset_name = 'dim_ml_dataset'
new_table_name = 'source__ieee_raw'
file_path = f'../input/{new_table_name}.csv'

# GCS
pj_name = 'gixo-dim'
bucket_name = 'horikoshi'

bq = BigQuery(dataset_name=dataset_name)

Setup Dataset dim_ml_dataset.


In [110]:
column_names = list(rawdata.columns)
column_types = []
for col in rawdata.dtypes:
    if str(col).count('int'):
        column_types.append('NUMERIC')
    elif str(col).count('float'):
        column_types.append('FLOAT')
    else:
        column_types.append('STRING')
column_modes = ['NULLABLE'] * len(column_names)

schema = bq.create_schema(column_names, column_types, column_modes)
bq.del_table(new_table_name)
bq.create_table(new_table_name, schema)

del table: TableReference(DatasetReference('gixo-dim', 'dim_ml_dataset'), 'source__ieee_raw') | Res: None
Table source__ieee_raw created.


In [111]:
from google.cloud import storage as gcs

storage_client = gcs.Client.from_service_account_json(project=pj_name, json_credentials_path=f'{HOME}/privacy/gixo-dim-horikoshi-all-gcp.json')

bucket = storage_client.get_bucket(bucket_name)
blob = bucket.blob(new_table_name)
blob.upload_from_filename(filename=file_path)

blob_name = new_table_name
bq.insert_from_gcs(new_table_name, bucket_name, blob_name)